# Point-in-Polygon Queries with GeoPandas and H3

In [ ]:
import geopandas as gpd
from h3 import h3

The data is contained as a shapefile ASAM_events.shp which is inside a folder ASAM_data_download

In [ ]:
zipfile = 'zip://ASAM_shp.zip!ASAM_data_download/ASAM_events.shp'
incidents = gpd.read_file(zipfile)
incidents

H3 library has 15 levels of resolution. [This table](https://h3geo.org/#/documentation/core-library/resolution-table) shows details of each level. We choose Level 3 which results in a grid size of approximately 100km.

The function `lat_lng_to_h3` converts a location's coordinates into an H3 id of the chosen level. We add a column called `h3` with the H3 cell id for the point at level 3.

In [ ]:
h3_level = 3

def lat_lng_to_h3(row):
    return h3.geo_to_h3(row.geometry.y, row.geometry.x, h3_level)

incidents['h3'] = incidents.apply(lat_lng_to_h3, axis=1)
incidents

Since all points that fall in a grid cell will have the same id, we can simply aggregate all rows with the same grid id to find all points that fall in the grid polygon. 

We use Panda's `groupby` function on the `h3` column and add a new column `count` to the output with the number of rows for each H3 id.

In [ ]:
counts = incidents.groupby(['h3']).h3.agg('count').to_frame('count').reset_index()
counts

To visualize the results or export it to a GIS, we need to convert the H3 cell ids to a geometry. The `h3_to_geo_boundary` function takes a H3 key and returns a list of coordinates that form the hexagonal cell. Since GeoPandas uses `shapely` library for constructing geometries, we convert the list of coordinates to a shapely `Polygon` object. Note the optional second argument to the `h3_to_geo_boundary` function which we have set to `True` which returns the coordinates in the *(x,y)* order compared to default *(lat,lon)*

In [ ]:
from shapely.geometry import Polygon

def add_geometry(row):
    points = h3.h3_to_geo_boundary(row['h3'], True)
    return Polygon(points)

counts['geometry'] = counts.apply(add_geometry, axis=1)
counts

We turn the dataframe to a GeoDataframe with the CRS EPSG:4326 and write it to a geopackage.

In [ ]:
gdf = gpd.GeoDataFrame(counts, crs='EPSG:4326')
output_filename = 'gridcounts.gpkg'
gdf.to_file(driver='GPKG', filename=output_filename)